In [1]:
import os
import glob
import gzip

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm

import shapely.wkt as wkt
from shapely.geometry import Point, LineString, box
from shapely.ops import nearest_points

import lxml.etree as ET
import pickle

import network_io as nio

# Define which we want to do
is_for_1pm = False
result_path = "this_is_an_example"

if is_for_1pm:
    base_dir = '../../../../data/pop_1pm_with_policies/'
    subdirs_pattern = os.path.join(base_dir, 'output_networks_*')
    subdirs = list(set(glob.glob(subdirs_pattern)))
    subdirs.sort()
    base_output_links_no_policies = '../../../data/pop_1pm/output_1pm/'
else:
    base_path = '../../../../data/pop_1pct_with_policies/output_networks_100_filtered/'
    network_folders = glob.glob(os.path.join(base_path, 'network_d_*'))
    base_output_links_no_policies = '../../../../data/pop_1pct/output_1pct/'

districts = gpd.read_file("../../../../data/visualisation/districts_paris.geojson")

# Abstract

Execute this notebook for a first processing of the outputs of the simulation. For further processing by GNNs, execute the notebook convert_data_to_output_for_gnn.

In [2]:
# def create_policy_key(folder_name):
#     # Extract the relevant part of the folder name
#     parts = folder_name.split('_')[1:]  # Ignore the first part ('network')
#     district_info = '_'.join(parts).replace('d_', '')
#     districts = district_info.split('_')
#     return f"policy introduced in Arrondissement(s) {', '.join(districts)}"
    
def create_policy_key(folder_name):
    # Extract the relevant part of the folder name
    base_name = os.path.basename(folder_name)  # Get the base name of the file or folder
    parts = base_name.split('_')[1:]  # Ignore the first part ('network')
    district_info = '_'.join(parts)
    districts = district_info.split('_')
    return f"Policy introduced in Arrondissement(s) {', '.join(districts)}"
    
# Plotting function
def plot_policy_data(key, df):
    arrondissement_number = key.replace('policy_', '').replace('_', ' ')
    arrondissement_number_cleaned = arrondissement_number.replace(" ", "_")

    print(f"Policy: {key}")
    
    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:2154")
    gdf = gdf.to_crs(epsg=4326)

    x_min = gdf.total_bounds[0] + 0.05
    y_min = gdf.total_bounds[1] + 0.05
    x_max = gdf.total_bounds[2]
    y_max = gdf.total_bounds[3]
    bbox = box(x_min, y_min, x_max, y_max)
    
    # Filter the network to include only the data within the bounding box
    gdf = gdf[gdf.intersects(bbox)]
    
    # Set up the plot
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    gdf[gdf['vol_car'] == 0].plot(ax=ax, color='lightgrey', linewidth=0.2, label='Network', zorder = 1)

    # Plot links with activity_count using the Viridis color scheme and log scale
    gdf.plot(column='vol_car', cmap='coolwarm', linewidth=1.5, ax=ax, legend=True,
             norm=LogNorm(vmin=gdf['vol_car'].min() + 1, vmax=gdf['vol_car'].max()),
             legend_kwds={'label': "Car volume", 'orientation': "vertical", 'shrink': 0.5})
    
    # districts.plot(ax=ax, facecolor='None', edgecolor='black', linewidth=1, label = "Arrondissements", zorder=3)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    
    # Customize the plot
    plt.title(f"{arrondissement_number}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.legend()
    plt.savefig("results/network_policies_new/" + f"{arrondissement_number_cleaned}.png", dpi=300, bbox_inches='tight')
    # plt.show()
    
# Function to iterate over result_dic and perform the required operations
def analyze_geodataframes(result_dic):
    # Extract the base network data for comparison
    base_gdf = result_dic.get("base_network_no_policies")
    if base_gdf is not None:
        base_vol_car = base_gdf['vol_car'].sum()
        base_gdf_car = base_gdf[base_gdf['modes'].str.contains('car')]
        base_capacity_car = base_gdf_car['capacity'].sum() * 0.005
        base_freespeed_car = base_gdf_car['freespeed'].sum()

    for policy, gdf in result_dic.items():
        print(f"Policy: {policy}")
        
        # Filter edges where 'modes' contains 'car'
        gdf_car = gdf[gdf['modes'].str.contains('car')]
        total_capacity_car = round(gdf_car['capacity'].sum() * 0.005)
        print(f"Total capacity of edges with 'car' mode: {total_capacity_car}")
        
        total_freespeed_car = round(gdf_car['freespeed'].sum())
        print(f"Total freespeed of edges with 'car' mode: {total_freespeed_car}")

        total_vol_car = gdf['vol_car'].sum()
        print(f"Total 'vol_car': {total_vol_car}")

        if policy != "base_network_no_policies" and base_gdf is not None:
            vol_car_increase = ((total_vol_car - base_vol_car) / base_vol_car) * 100
            capacity_car_increase = ((total_capacity_car - base_capacity_car) / base_capacity_car) * 100
            freespeed_car_increase = ((total_freespeed_car - base_freespeed_car) / base_freespeed_car) * 100

            print(f"Percentage increase in 'vol_car': {vol_car_increase:.2f}%")
            print(f"Percentage increase in capacity (car edges): {capacity_car_increase:.2f}%")
            print(f"Percentage increase in freespeed (car edges): {freespeed_car_increase:.2f}%")
        
        print()
        
        
# Function to read and convert CSV.GZ to GeoDataFrame
def read_network_data(folder):
    file_path = os.path.join(folder, 'output_links.csv.gz')
    if os.path.exists(file_path):
        # Read the CSV file with the correct delimiter
        df = pd.read_csv(file_path, delimiter=';')
        # Convert the 'geometry' column to actual geometrical data
        df['geometry'] = df['geometry'].apply(wkt.loads)
        
        # Create a GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='geometry')
        return gdf
    else:
        return None
    

In [3]:
# Read all network data into a dictionary of GeoDataFrames
result_dic = {}

if is_for_1pm:
    # Loop through each subdirectory
    counter = 0
    result_dic = {}

    for subdir in subdirs:
        print(f'Accessing folder: {subdir}')
        print(len(os.listdir(subdir)))
        for network in os.listdir(subdir):
            folder = os.path.join(subdir, network)
            file_path = os.path.join(subdir, network)
            if file_path.endswith(".DS_Store"):
                continue
            counter += 1
            policy_key = create_policy_key(network)
            gdf = read_network_data(file_path)
            if gdf is not None:
                result_dic[policy_key] = gdf
                counter += 1
else:
    for folder in network_folders:
        policy_key = create_policy_key(os.path.basename(folder))
        gdf = nio.read_network_data(folder)
        if gdf is not None:
            result_dic[policy_key] = gdf
        
base_network_no_policies = nio.read_network_data(base_output_links_no_policies)
result_dic["base_network_no_policies"] = base_network_no_policies

In [4]:
for key, df in result_dic.items():
    print(key)
    # plot_policy_data(key, df)

Policy introduced in Arrondissement(s) d, 13
Policy introduced in Arrondissement(s) d, 14
Policy introduced in Arrondissement(s) d, 15
Policy introduced in Arrondissement(s) d, 12
Policy introduced in Arrondissement(s) d, 1
Policy introduced in Arrondissement(s) d, 10
Policy introduced in Arrondissement(s) d, 11
Policy introduced in Arrondissement(s) d, 16
base_network_no_policies


In [5]:
analyze_geodataframes(result_dic)

Policy: Policy introduced in Arrondissement(s) d, 13
Total capacity of edges with 'car' mode: 159199
Total freespeed of edges with 'car' mode: 257901
Total 'vol_car': 1614235
Percentage increase in 'vol_car': -0.53%
Percentage increase in capacity (car edges): -2.18%
Percentage increase in freespeed (car edges): -0.00%

Policy: Policy introduced in Arrondissement(s) d, 14
Total capacity of edges with 'car' mode: 159834
Total freespeed of edges with 'car' mode: 257901
Total 'vol_car': 1622768
Percentage increase in 'vol_car': -0.01%
Percentage increase in capacity (car edges): -1.79%
Percentage increase in freespeed (car edges): -0.00%

Policy: Policy introduced in Arrondissement(s) d, 15
Total capacity of edges with 'car' mode: 159245
Total freespeed of edges with 'car' mode: 257901
Total 'vol_car': 1616852
Percentage increase in 'vol_car': -0.37%
Percentage increase in capacity (car edges): -2.16%
Percentage increase in freespeed (car edges): -0.00%

Policy: Policy introduced in Arron

## Save the results_dic, so that we can use it for ML methods

In [6]:
# Save the dictionary
# with open('results/result_path', 'wb') as f:
#     pickle.dump(result_dic, f)